In [ ]:
!pip install --upgrade sympy
!pip install --force-reinstall sympy

In [2]:
import random
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    get_scheduler
)
from torch.optim import AdamW
from transformers.optimization import SchedulerType
!pip install datasets
from datasets import load_dataset
from tqdm import tqdm


In [1]:

# Fix the random seed for reproducibility
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Multitask Model Wrapper
class MultitaskModelWrapper:
    def __init__(self, task_configs):
        """
        Initialize multitask wrapper for different tasks.
        Args:
            task_configs (dict): Configuration for tasks including model and tokenizer.
        """
        self.models = {}
        self.tokenizers = {}
        for task_name, config in task_configs.items():
            self.models[task_name] = AutoModelForSequenceClassification.from_pretrained(
                config["model_name"],
                num_labels=config["num_labels"]
            )
            self.tokenizers[task_name] = AutoTokenizer.from_pretrained(config["model_name"])
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        for model in self.models.values():
            model.to(self.device)

    def train(self, dataloaders, epochs=3, lr=5e-5, weight_decay=0.01):
        """
        Train models for all tasks.
        Args:
            dataloaders (dict): Dataloaders for tasks.
            epochs (int): Number of epochs.
            lr (float): Learning rate.
            weight_decay (float): Weight decay for AdamW.
        """
        optimizers = {
            task: AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
            for task, model in self.models.items()
        }
        schedulers = {
            task: get_scheduler(
                name=SchedulerType.LINEAR,
                optimizer=optimizers[task],
                num_warmup_steps=0,
                num_training_steps=epochs * len(dataloaders[task])
            )
            for task in self.models.keys()
        }

        for epoch in range(epochs):
            print(f"Epoch {epoch + 1}/{epochs}")
            for task, model in self.models.items():
                model.train()
                total_loss = 0
                for batch in tqdm(dataloaders[task], desc=f"Training {task}"):
                    inputs = {k: v.to(self.device) for k, v in batch.items() if k != "labels"}
                    labels = batch["labels"].to(self.device)

                    optimizers[task].zero_grad()
                    outputs = model(**inputs, labels=labels)
                    loss = outputs.loss
                    loss.backward()
                    optimizers[task].step()
                    schedulers[task].step()
                    total_loss += loss.item()
                print(f"{task} - Loss: {total_loss / len(dataloaders[task])}")

    def evaluate(self, dataloaders):
        """
        Evaluate models on all tasks.
        Args:
            dataloaders (dict): Evaluation dataloaders.
        Returns:
            dict: Evaluation results for each task.
        """
        results = {}
        for task, model in self.models.items():
            model.eval()
            correct, total = 0, 0
            for batch in tqdm(dataloaders[task], desc=f"Evaluating {task}"):
                inputs = {k: v.to(self.device) for k, v in batch.items() if k != "labels"}
                labels = batch["labels"].to(self.device)

                with torch.no_grad():
                    outputs = model(**inputs)
                preds = torch.argmax(outputs.logits, dim=-1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
            results[task] = correct / total
        return results



In [6]:

# Task Configurations
task_configs = {
    "sentiment": {"model_name": "bert-base-uncased", "num_labels": 5},
    "paraphrase": {"model_name": "roberta-base", "num_labels": 2},
    # "similarity": {"model_name": "distilbert-base-uncased", "num_labels": 1},
}

# Load Data (Example using Hugging Face's datasets library)
from datasets import load_dataset
from transformers import AutoTokenizer
from torch.utils.data import DataLoader

def load_data(tokenizer, split="train", max_samples=1000, task_name=None):

    dataset = load_dataset("glue", "sst2" if split == "train" else "qqp", split=split)
    dataset = dataset.select(range(min(max_samples, len(dataset))))  # Reduce size

    # Preprocessing function
    def preprocess_function(examples):
        tokenized = tokenizer(examples["sentence"], padding="max_length", truncation=True)
        return tokenized

    dataset = dataset.map(preprocess_function, batched=True)

    # Adjust label dtype based on task
    if task_name == "similarity":  # For regression, use Float
        dataset = dataset.map(lambda x: {"labels": float(x["label"])})
    else:  # For classification, use Long
        dataset = dataset.rename_column("label", "labels")  # Rename column
        dataset = dataset.map(lambda x: {"labels": int(x["labels"])})

    dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
    return dataset



dataloaders = {}
for task_name, config in task_configs.items():
    tokenizer = AutoTokenizer.from_pretrained(config["model_name"])
    data = load_data(tokenizer, split="train", max_samples=500, task_name=task_name)
    dataloaders[task_name] = DataLoader(data, batch_size=16, shuffle=True)




In [7]:
# Training and Evaluation
if __name__ == "__main__":
    seed_everything()
    multitask_model = MultitaskModelWrapper(task_configs)
    multitask_model.train(dataloaders, epochs=3)
    results = multitask_model.evaluate(dataloaders)
    print("Evaluation Results:", results)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3


Training sentiment: 100%|██████████| 32/32 [00:40<00:00,  1.27s/it]


sentiment - Loss: 0.883944833651185


Training paraphrase: 100%|██████████| 32/32 [00:41<00:00,  1.30s/it]


...

sentiment - Loss: 0.18095844145864248


Training paraphrase: 100%|██████████| 32/32 [00:44<00:00,  1.39s/it]


paraphrase - Loss: 0.32627327437512577


Evaluating paraphrase: 100%|██████████| 32/32 [00:13<00:00,  2.42it/s]

Evaluation Results: {'sentiment': 0.988, 'paraphrase': 0.93}


In [8]:
for task, result in results.items():
  print(f"Task: {task}, Result: {result}")

Task: sentiment, Result: 0.988
Task: paraphrase, Result: 0.93
